# JWST Imaging Pipeline

Prepared for AAS Meeting JWebbinar

**Consolidator**: Karl Gordon (kgordon@stsci.edu)| **Latest Update**: 20 Dec 2021

Completed based on the excellent and expansive JWebbinar 3 notebooks written by Bryan Hilbert (hilbert@stsci.edu).

Please see the JWebbinar 3, 4, and 5 notebooks for more in depth explanations and other ways of running the JWST pipeline.

JWebbinar info: https://www.stsci.edu/jwst/science-execution/jwebbinars

## Notebook Goals

Run the JWST pipeline for a set of NIRCam images.

Use only one of the methods for running the pipeline, specifically running a stage.
See the JWebbinar 3 notebooks for ways to run the pipeline step-by-step for each stage or from the commandline.

### Imports and setup

In [ ]:
# Packages that allow us to get information about objects:
from glob import glob
import asdf
import copy
import os
import shutil

# Numpy library:
import numpy as np

# For downloading data
import requests

# To read association file
import json

# Astropy tools:
from astropy.io import ascii, fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch

Set the location to cache pipeline reference files

In [ ]:
os.environ["CRDS_PATH"] = "./crds_cache"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
#%matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

In [ ]:
# List of possible data quality flags
from jwst.datamodels import dqflags

# import the JWST datamodels
from jwst import datamodels

# entire pipelines pipeline
from jwst.pipeline import calwebb_detector1
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_image3

In [ ]:
import jwst
print(jwst.__version__)

## Define convenience functions and parameters

In [ ]:
# To make everything easier, all files saved by the pipeline
# and pipeline steps will be saved to the working directory.
# This will be more important for the level 2 and 3 pipelines
# once we are working with association files.
output_dir = './'

In [ ]:
# Make sure the output directory exists before downloading any data
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
def download_files(files, output_directory, force=False):
    """Given a tuple or list of tuples containing (URL, filename),
    download the given files into the current working directory.
    Downloading is done via astropy's download_file. A symbolic link
    is created in the specified output dirctory that points to the
    downloaded file.
    
    Parameters
    ----------
    files : tuple or list of tuples
        Each 2-tuple should contain (URL, filename), where
        URL is the URL from which to download the file, and
        filename will be the name of the symlink pointing to
        the downloaded file.
        
    output_directory : str
        Name of the directory in which to create the symbolic
        links to the downloaded files
        
    force : bool
        If True, the file will be downloaded regarless of whether
        it is already present or not.
                
    Returns
    -------
    filenames : list
        List of filenames corresponding to the symbolic links
        of the downloaded files
    """
    # In the case of a single input tuple, make it a
    # 1 element list, for consistency.
    filenames = []
    if isinstance(files, tuple):
        files = [files]
        
    for file in files:
        filenames.append(file[1])
        if force:
            print('Downloading {}...'.format(file[1]))
            demo_file = download_file(file[0], cache='update')
            # Make a symbolic link using a local name for convenience
            if not os.path.islink(os.path.join(output_directory, file[1])):
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
        else:
            if not os.path.isfile(os.path.join(output_directory, file[1])):
                print('Downloading {}...'.format(file[1]))
                demo_file = download_file(file[0], cache=True)
                # Make a symbolic link using a local name for convenience
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
            else:
                print('{} already exists, skipping download...'.format(file[1]))
                continue
    return filenames    

In [ ]:
def show_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None,
               scale='log', units='MJy/str'):
    """Function to generate a 2D, log-scaled image of the data, 
    with an option to highlight a specific pixel.
    
    data_2d : numpy.ndarray
        2D image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    xpixel : int
        X-coordinate of pixel to highlight
        
    ypixel : int
        Y-coordinate of pixel to highlight
        
    title : str
        String to use for the plot title
        
    scale : str
        Specify scaling of the image. Can be 'log' or 'linear'
        
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    if scale == 'log':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    elif scale == 'linear':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LinearStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    fig.colorbar(im, label=units)
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

In [ ]:
def overlay_catalog(data_2d, catalog, flux_limit=0, vmin=0, vmax=10,
                    title=None, units='MJy/str'):
    """Function to generate a 2D image of the data, 
    with sources overlaid.
    
    data_2d : numpy.ndarray
        2D image to be displayed
        
    catalog : astropy.table.Table
        Table of sources
    
    flux_limit : float
        Minimum signal threshold to overplot sources from catalog.
        Sources below this limit will not be shown on the image.
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    title : str
        String to use for the plot title
                
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    for row in catalog:
        if row['aper_total_flux'].value > flux_limit:
            plt.plot(row['xcentroid'], row['ycentroid'], marker='o',
                     markersize='3', color='red')

    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    fig.colorbar(im, label=units)
    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    
    if title:
        plt.title(title)

## Download Data

In [ ]:
uncal_info = ('https://stsci.box.com/shared/static/j46wpyirlbqo30e7c9719ycnuc1qk2lu.fits',
              'jw98765001001_01101_00003_nrcb5_uncal.fits')
uncal_file = download_files(uncal_info, output_dir, force=False)[0]

In [ ]:
# Also download a "trapsfilled" file, which specifies the state of the charge traps at
# the time of the preceding exposure. This will serve as input to the persistence
# step.
persist_info = ('https://stsci.box.com/shared/static/ehkof12d43h6nnpijs2r4tyuzde3nzc9.fits',
                'jw98765001001_01101_00002_nrcb5_trapsfilled.fits')
persist_file = download_files(persist_info, output_dir, force=False)[0]

## CALWEBB_DETECTOR1

Full notebook for CALWEBB_DETECTOR1: https://stsci.app.box.com/s/z5bznws56f9m1j505vhnpud35nxrjr50

In [ ]:
# Create an instance of the pipeline class
detector1 = calwebb_detector1.Detector1Pipeline()

# Set some parameters that pertain to the
# entire pipeline
detector1.output_dir = output_dir
detector1.save_results = True

# Set some parameters that pertain to some of
# the individual steps
detector1.refpix.use_side_ref_pixels = True
detector1.linearity.save_results = True
detector1.jump.rejection_threshold = 6

# Specify the name of the trapsfilled file, which
# contains the state of the charge traps at the end
# of the preceding exposure
detector1.persistence.input_trapsfilled = persist_file

# Call the run() method
run_output = detector1.run(uncal_file)

### Examine the outputs

In [ ]:
# Generate the rate file name from the uncal file name
rate_file = uncal_file.replace('uncal.fits', 'rate.fits')

In [ ]:
rate_file

In [ ]:
# Use getdata to quickly read in the science data from the rate file
rate_data = fits.getdata(rate_file)

In [ ]:
# Look at the rate image
show_image(rate_data, 0.5, 10, units="DN")

## CALWEBB_IMAGE2

Full notebook for CALWEBB_IMAGE2: https://stsci.app.box.com/s/8rulrbsumk3umh1w90k4cruwrsrtwsvo

## Download Data

In [ ]:
nircam_info = [('https://stsci.box.com/shared/static/g6316wjr4mv936rlouzdjeq065s7ou6g.fits',
                'jw98765001001_01101_00001_nrcb5_rate.fits'),
               ('https://stsci.box.com/shared/static/z2xunff1d2g3m3fjxc1fixoz8rjfpl7h.fits',
                'jw98765001001_01101_00002_nrcb5_rate.fits'),
               ('https://stsci.box.com/shared/static/4xuvt56kr7gix7dx3tntek6wc9kockef.fits',
                'jw98765001001_01101_00003_nrcb5_rate.fits'),
               ('https://stsci.box.com/shared/static/lzhcnzds2l7mpf92oet1u69uof788u3l.json',
                'level2_lw_asn.json'),
               ('https://stsci.box.com/shared/static/d4pu8ieyjc27wzoe0of3ajb9vjtvc80g.asdf',
                'image2_pipeline_params.asdf')]
nircam_files = download_files(nircam_info, output_dir, force=False)

In [ ]:
# the image2 pipeline is usually run on a set of images defined in an association file
asn_file = os.path.join(output_dir, 'level2_lw_asn.json')

In [ ]:
# Create an instance of the pipeline class
image2 = calwebb_image2.Image2Pipeline()

# Set some parameters that pertain to the
# entire pipeline
image2.output_dir = output_dir
image2.save_results = True

# Set some parameters that pertain to some of
# the individual steps
image2.resample.pixfrac = 1.0    # this is the default. Set here as an example

# Call the run() method
image2.run(asn_file)

### Examine the outputs

In [ ]:
# Get a list of input file names from the association file
with open(asn_file) as f_obj:
  asn_data = json.load(f_obj)
input_files = [item['members'][0]['expname'] for item in asn_data['products']]  

In [ ]:
# Get a list of the output file names
output_files = sorted(glob(os.path.join(output_dir, '*_cal.fits')))

In [ ]:
# Show the first calibrated output file
cal_data = fits.open(output_files[0])
show_image(cal_data['SCI'].data, 0.15, 5)
cal_data.close()

In [ ]:
# Show the second calibrated output file
cal_data = fits.open(output_files[1])
show_image(cal_data['SCI'].data, 0.15, 5)
cal_data.close()

In [ ]:
# Show the third calibrated output file
cal_data = fits.open(output_files[2])
show_image(cal_data['SCI'].data, 0.15, 5)
cal_data.close()

## CALWEBB_IMAGE3

Full notebook for CALWEBB_IMAGE3: https://stsci.app.box.com/s/ausbdbmbfi7dhu7yuo4zl56f9ex5qt4b

## Download Data

In [ ]:
nircam_info = [('https://stsci.box.com/shared/static/7d00b9isvss7njhcwmd8uiq8c7s4d845.json',
                'level3_lw_asn.json'),
               ('https://stsci.box.com/shared/static/ja0gkd8c0x8p8konhr84wkuhwnpkqf4s.asdf',
                'jwst_nircam_pars-tweakregstep_0006.asdf'),
               ('https://stsci.box.com/shared/static/yahdw55fotwrh7i6hhcxksj97qkf7j4r.asdf',
                'nircam_pars-sourcecatalogstep_f444w_clear.asdf')
              ]
nircam_files = download_files(nircam_info, output_dir, force=False)

In [ ]:
# the image2 pipeline is run on a set of images defined in an association file
asn_file = os.path.join(output_dir, 'level3_lw_asn.json')

In [ ]:
# Create an instance of the pipeline class
image3 = calwebb_image3.Image3Pipeline()

# Set some parameters that pertain to the
# entire pipeline
image3.output_dir = output_dir
image3.save_results = True

# Set some parameters that pertain to some of
# the individual steps
image3.tweakreg.snr_threshold = 10.0  # 5.0 is the default
image3.tweakreg.kernel_fwhm = 2.302  # 2.5 is the default
image3.tweakreg.brightest = 20  # 100 is the default
image3.source_catalog.kernel_fwhm = 2.302  # pixels
image3.source_catalog.snr_threshold = 10.

# Call the run() method
image3.run(asn_file)

### Examine the outputs

In [ ]:
# Open the association file and load into a json object
with open(asn_file) as f_obj:
  asn_data = json.load(f_obj)
input_files = [item['expname'] for item in asn_data['products'][0]['members']]  

mosaic_file = os.path.join(output_dir, 'l3_lw_results_i2d.fits')
source_cat_file = os.path.join(output_dir, 'l3_lw_results_cat.ecsv')
segmentation_map_file = os.path.join(output_dir, 'l3_lw_results_segm.fits')
cr_flagged_files = [item.replace('cal.fits', 'crf.fits') for item in input_files]

Read in the final mosaic image and display

Note that there are low level residual horizontal instrument artifacts present in the final mosaic.

In [ ]:
mosaic = datamodels.open(mosaic_file)
show_image(mosaic.data, vmin=-0, vmax=5)

Let's look at the segmentation map that was created by the `source_catalog` step. This shows which pixels are associated with the identified sources.

In [ ]:
seg_map = fits.getdata(segmentation_map_file)
show_image(seg_map, vmin=0, vmax=5, scale='linear')

And now examine the actual source catalog. For each source, the catalog lists the location, along with flux and AB/Vega magnitude values in three different apertures, as well as calculated values for an infinite aperture. Within the documentation, you can see the [full list of column definitions](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html#source-catalog-table).

In [ ]:
source_cat = ascii.read(source_cat_file)
source_cat

Finally, let's overlay the source catalog on top of the mosaic image. In order to cut down on the number of spurious detections, we only show sources above a minimum flux limit. Another way to cut down on the number of spurious detections would be to change some of the `source_catalog` parameter values when calling the pipeline above.

In [ ]:
overlay_catalog(mosaic.data, source_cat, flux_limit=5e-7, vmin=0, vmax=10,
                title='Final mosaic with source catalog')